# Credit Risk Resampling Techniques

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [4]:
# Create our features
X = df.iloc[:, :-1]

# Create our target
y = df.iloc[:, -1]

In [5]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [6]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

**Encoding Categorical Data**






Ecoding the Independent Varible

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['homeowner'])], remainder='passthrough')

X = np.array(ct.fit_transform(X)) #T

In [8]:
print(X[0])

[0.00000000e+00 1.00000000e+00 0.00000000e+00 1.07000000e+04
 7.67200000e+00 5.28000000e+04 4.31818182e-01 5.00000000e+00
 1.00000000e+00 2.28000000e+04]


Encoding the Dependent Variable

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [10]:
# Create X_train, X_test, y_train, y_test

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [11]:
print(X_train)

[[0.00e+00 1.00e+00 0.00e+00 ... 3.00e+00 0.00e+00 1.71e+04]
 [0.00e+00 1.00e+00 0.00e+00 ... 4.00e+00 1.00e+00 2.14e+04]
 [0.00e+00 1.00e+00 0.00e+00 ... 4.00e+00 0.00e+00 1.98e+04]
 ...
 [0.00e+00 1.00e+00 0.00e+00 ... 4.00e+00 1.00e+00 2.08e+04]
 [1.00e+00 0.00e+00 0.00e+00 ... 5.00e+00 1.00e+00 2.44e+04]
 [0.00e+00 1.00e+00 0.00e+00 ... 5.00e+00 1.00e+00 2.24e+04]]


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [12]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()



In [13]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_train[:,3:] = sc.fit_transform(X_train[:,3:])

In [14]:
# Scale the training and testing data

X_test[:,3:] = sc.transform(X_test[:,3:])

# Simple Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.988357778120512

In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  498,     9],
       [   83, 14918]])

In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.98      0.99      0.92      0.99      0.98       507
          1       1.00      0.99      0.98      1.00      0.99      0.98     15001

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     15508



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [19]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler 
ros = RandomOverSampler(random_state=0)
X_res, y_res = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
print("Original Dataset {}".format(Counter(y_train)))
print("Resampled Dataset {}".format(Counter(y_res)))


Original Dataset Counter({1: 60035, 0: 1993})
Resampled Dataset Counter({1: 60035, 0: 60035})


In [20]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_res, y_res)

LogisticRegression(random_state=1)

In [21]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9939749578824922

In [22]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  504,     3],
       [   92, 14909]])

In [23]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.99      0.99      0.91      0.99      0.99       507
          1       1.00      0.99      0.99      1.00      0.99      0.99     15001

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     15508



### SMOTE Oversampling

In [24]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
method = SMOTE(random_state = 0)
X_resampled, y_resampled = method.fit_resample(X_train, y_train)

# View the count of target classes with Counter
print("Original Dataset {}".format(Counter(y_train)))
print("Resampled Dataset {}".format(Counter(y_resampled)))

Original Dataset Counter({1: 60035, 0: 1993})
Resampled Dataset Counter({1: 60035, 0: 60035})


In [25]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [26]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9940082889937515

In [27]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  504,     3],
       [   91, 14910]])

In [28]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.99      0.99      0.91      0.99      0.99       507
          1       1.00      0.99      0.99      1.00      0.99      0.99     15001

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     15508



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [29]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(sampling_strategy={0: 10})
X_cc, y_cc = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
print("Original Dataset {}".format(Counter(y_train)))
print("Resampled Dataset {}".format(Counter(y_cc)))

Original Dataset Counter({1: 60035, 0: 1993})
Resampled Dataset Counter({1: 60035, 0: 10})


In [30]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_cc, y_cc)

LogisticRegression(random_state=1)

In [31]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5

In [32]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[    0,   507],
       [    0, 15001]])

In [33]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00       507
          1       0.97      1.00      0.00      0.98      0.00      0.00     15001

avg / total       0.94      0.97      0.03      0.95      0.00      0.00     15508



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [34]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sme = SMOTEENN(random_state=0)
X_sme, y_sme = sme.fit_resample(X_train, y_train)
# View the count of target classes with Counter
print("Original Dataset {}".format(Counter(y_train)))
print("Resampled Dataset {}".format(Counter(y_sme)))

Original Dataset Counter({1: 60035, 0: 1993})
Resampled Dataset Counter({1: 59629, 0: 59375})


In [35]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_sme, y_sme)

LogisticRegression(random_state=1)

In [36]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)
0.9940082889937515

0.9940082889937515

In [37]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  504,     3],
       [   91, 14910]])

In [38]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.99      0.99      0.91      0.99      0.99       507
          1       1.00      0.99      0.99      1.00      0.99      0.99     15001

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     15508



# Final Questions

1. Which model had the best balanced accuracy score?

   SMOTE and SMOTEEN

2. Which model had the best recall score?

    Almost all models had 99% recall scores except for LogisticRegression model that had 97% recall score.